In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import load_model
classifier = load_model('thaiseg_300krows_8timestep_bidirection.h5')


Using TensorFlow backend.


In [2]:
# Importing the training set
df = pd.read_csv('timeStep8CorpusFull.csv',encoding='utf-8')
print(df.shape)
df = df.head(300000)
df.head()

(1172847, 10)


,Unnamed: 0,0,1,2,3,4,5,6,7,isEnd
0,0,ม,ี,ค,ำ,ถ,า,ม,ม,0
1,1,ี,ค,ำ,ถ,า,ม,ม,า,0
2,2,ค,ำ,ถ,า,ม,ม,า,ก,0
3,3,ำ,ถ,า,ม,ม,า,ก,ม,0
4,4,ถ,า,ม,ม,า,ก,ม,า,0


In [3]:
#X and y
X = df[['0','1','2','3','4','5','6','7']]
y = df[['isEnd']]

#Label each character
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X[['0']])
X = X.apply(le.transform)
maxChar = 65

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
timeStep=8
def thaiSegment(paragraph):
    sentences = paragraph.split(" ")
    all_words = []
    for sentence in sentences:
        test = sentence[-timeStep+1:]+sentence #Same trick as in CNN
        df = {}
        for i in range(timeStep):
            if(i-timeStep+1<0):
                df[i] = list(test[i:i-timeStep+1])
            else:
                df[i] = list(test[i:])
        df_test = pd.DataFrame(df)
        df_test = df_test.apply(le.transform).values
        df_test = (np.arange(maxChar) == df_test[:,:,None]-1).astype(int)
        y_pred = classifier.predict(df_test)
        y_pred = y_pred>0.5

        df_test  = pd.DataFrame(df)
        df_test['isEnd'] = y_pred

        words = []
        w = ''

        for i in range(len(test)-timeStep+1):
            c = test[i+timeStep-1]
            w+= c
            if(y_pred[i][0]==True):
                words.append(w)
                w=''

        words.append(w)
        if "" in words: 
            words.remove('')
        all_words.append("|".join(words))
    return " ".join(all_words)


In [22]:
def compareToken2Real(tokenized,real):
    test = tokenized
    correct = 0
    incorrect = 0
    i=0
    j=0
    if(len(tokenized)<200): print(tokenized)
    cm = [[0,0],[0,0]]
    print("Start comparing")
    while(i<len(real) and j<len(test)):
        if(real[i]=="|" and test[j]=="|"):#True Positive
            cm[0][0]+=1
            i+=1
            j+=1
        elif(real[i]==test[j]):#True Negative
            cm[1][1]+=1
            i+=1
            j+=1
        elif(real[i]=="|"):#False Negative
            cm[0][1]+=1
            i+=2
            j+=1
        elif(test[j]=="|"):#False Positive
            cm[1][0]+=1
            j+=2
            i+=1
    n = cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0]
    acc = (cm[0][0]+cm[1][1])/n
    precision = cm[0][0]/(cm[0][0]+cm[1][0])
    recall = cm[0][0]/(cm[0][0]+cm[0][1])
    n = cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0]
    acc = (cm[0][0]+cm[1][1])/n
    precision = cm[0][0]/(cm[0][0]+cm[1][0])
    recall = cm[0][0]/(cm[0][0]+cm[0][1])
    
    print("Accuracy : {:0.2f}".format(acc*100))
    print("Precision : {:0.2f}".format(precision*100))
    print("Recall : {:0.2f}".format(recall*100))
    print("F1 Score : {:0.2f}".format(((2*precision*recall)/(precision+recall))*100))

In [23]:
def compareTokenizerScore(sentence='สวัสดี|ครับ|ผม|ชื่อ|เคอิโงะ'):
    raw_sentence = sentence.replace("|","")

    tokenized4 = thaiSegment(raw_sentence)


    compareToken2Real(tokenized4,sentence)
    
    
compareTokenizerScore('ปกปิด|ข้อมูล|การ|ระบาด|ของ|โรค|ต่อ|เกษตรกร|จน|ทำ|ให้|ไก่|ที่|เกษตรกร|เลี้ยง|ไว้|เป็น|ราย|ได้|หลัก|ของ|ครัวเรือน|ต้อง|ตาย|เป็น|จำนวน|มหาศาล')

ปกปิด|ข้อมูล|การ|ระบาด|ของ|โรคต่อเกษตรกรจน|ทำ|ให้|ไก่|ที่|เกษตรกรเลี้ยง|ไว้|เป็น|ราย|ได้|หลัก|ของ|ครัว|เรือน|ต้องตาย|เป็น|จำนวน|มหาศาล
Start comparing
Accuracy : 95.49
Precision : 95.45
Recall : 80.77
F1 Score : 87.50


In [12]:
#Compare with other tokenizer
def convertLetter(row):
    return row['7'] if row['isEnd']==0 else row['7']+"|"
df['letter'] = df.apply(convertLetter,axis=1)
testString = "".join(df['letter'].values)

In [13]:
print("Test set 1")
compareTokenizerScore(testString[:100000])
print("Test set 2")
compareTokenizerScore(testString[100001:220000])
print("Test set 3")
compareTokenizerScore(testString[250001:300000])

Test set 1
Start comparing
Accuracy : 96.76
Precision : 90.18
Recall : 94.02
F1 Score : 92.06
Test set 2
Start comparing
Accuracy : 96.51
Precision : 90.35
Recall : 92.61
F1 Score : 91.47
Test set 3
Start comparing
Accuracy : 96.50
Precision : 90.79
Recall : 92.27
F1 Score : 91.52
